In [2]:
import pandas as pd

# import spacy
import numpy as np
import sklearn as skl 

In [9]:
target = 'label'
input_column = 'cleanText'

train_data = pd.read_csv('../../../Files/Submissions/train/train2.csv', sep='|') 
# valid_data = pd.read_pickle('../../../Files/Submissions/train/val_split_submission.pickle')
test_data = pd.read_csv('../../../Files/Submissions/train/test2.csv', sep='|')

train_data = train_data[[target, input_column]]
# valid_data = valid_data[[target, input_column]]
test_data = test_data[[target, input_column]]

data = pd.concat([train_data,  test_data])


train_instances = train_data[input_column].apply(str).apply(str.split)
train_labels = train_data[target]

# collect known word tokens and tags
wordset, labelset = set(), set()

# collect tags from all data, to prevent unseen labels
labelset.update(set(data[target]))

# get the vocabulary
for words in train_instances:
    wordset.update(set(words))

# map words and tags into ints
PAD = '-PAD-'
UNK = '-UNK-'
word2int = {word: i + 2 for i, word in enumerate(sorted(wordset))}
word2int[PAD] = 0  # special token for padding
word2int[UNK] = 1  # special token for unknown words
 
label2int = {label: i for i, label in enumerate(sorted(labelset))}
# inverted index to translate it back
int2label = {i:label for label, i in label2int.items()}


def convert2ints(instances):
    """
    function to apply the mapping to all words
    """
    result = []
    for words in instances:
        # replace words with int, 1 for unknown words
        word_ints = [word2int.get(word, 1) for word in words]
        result.append(word_ints)
    return result
                          
train_instances_int = convert2ints(train_instances)
train_labels_int = [label2int[label] for label in train_labels]

test_instances = test_data[input_column].apply(str).apply(str.split)
test_labels = test_data[target]

test_instances_int = convert2ints(test_instances)
test_labels_int = [label2int[label] for label in test_labels]

# convert dev data
# val_instances = valid_data[input_column].apply(str).apply(str.split)
# val_labels = valid_data[target]

# val_instances_int = convert2ints(val_instances)
# val_labels_int = [label2int[label] for label in val_labels]


In [10]:
train_labels.unique()

array([2, 1, 0])

In [11]:

# compute 95th percentile of training sentence lengths
L = sorted(map(len, train_instances))
MAX_LENGTH = L[int(len(L)*0.95)]

# apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_instances_int = pad_sequences(train_instances_int, padding='post', maxlen=MAX_LENGTH)
test_instances_int = pad_sequences(test_instances_int, padding='post', maxlen=MAX_LENGTH)
# val_instances_int = pad_sequences(val_instances_int, padding='post', maxlen=MAX_LENGTH)

In [12]:
from sklearn.linear_model import LogisticRegression 

In [13]:
lrf = LogisticRegression(random_state=42)

In [14]:
len(train_instances_int)

1471

In [15]:
lrf.fit(train_instances_int, train_labels_int)

/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [18]:
y_pred = lrf.predict(test_instances_int)

In [19]:
test_instances_int.shape

(368, 88)

In [20]:
y_pred.shape

(368,)

In [16]:
y_pred.tofile('../../../Files/models/lr_pred.csv', sep='\n')

In [13]:
test_labels.to_csv('../../../Files/models/y_true.csv', sep='\n', index=False)